In [ ]:
import matplotlib.pyplot as plt
import glob
import re
from Bio import PDB
from Bio.PDB.PDBIO import PDBIO
from Bio.PDB.StructureBuilder import StructureBuilder
import MDAnalysis as mda
import numpy as np
from MDAnalysis.analysis import distances

### Load Pesto and Scannet models (PDB format)

In [ ]:
Pesto_Scannet=["PeSTo/ea3169_interface_protein.pdb", "ScanNet/annotated_fold_banal_20_52_model_0.pdb"]

### Extract PPI score

In [ ]:
# prepare dictionary
PPI={}
for m in Pesto_Scannet:
    PPI[m]=[]
    # read Bfactors where PPI score is stored
    u = mda.Universe(m)
    # get Bfactors
    at_CA = u.select_atoms("name CA")
    PPI[m] = [ a.tempfactor for a in at_CA ]

### extract contacts from PDB

In [ ]:
PDB_ = "AF3_model.pdb"
u = mda.Universe(PDB_)
# get all CA atoms
CA_atoms = u.select_atoms('name CA')

# get the CA atoms of spike and from the antibody Fab domain. You must edit the chains that must be selected if necessary
spike_CA = CA_atoms.select_atoms('chainID A')
# get the CA atoms of the antibody Fab domain. This all the other chains wrt to the spike chain
antibody_CA = CA_atoms.select_atoms('not chainID A')

print("spike number of residues: {}".format(len(spike_CA)))
print("antibody number of residues: {}".format(len(antibody_CA)))
# calculate distance matrix
distance_matrix = distances.distance_array(spike_CA, antibody_CA)
# set cutoff
distance_cutoff = 8.0
# identify contacts
ids = np.where(distance_matrix < distance_cutoff)
# contact residues
contacts = sorted(list(set(ids[0])))
print(contacts)

### Plot per-residue PPI score and actual contacts in PDB

In [ ]:
# plot predictions
for key in PPI:
    l=re.sub(".pdb","",key)
    plt.plot(PPI[key], '-', label=l)
# plot real contacts
for c in contacts:
    plt.axvline(c, 0, 1, color="grey", linestyle="--")
plt.xlabel('Residue ID')
plt.ylabel('PPI score')
plt.legend(fontsize=8, frameon=False, ncols=1, loc="upper right")
plt.show()